In [51]:
import pandas

data = pandas.read_csv('salary-train.csv',nrows=10000)
data.head()

,FullDescription,LocationNormalized,ContractTime,SalaryNormalized
0,International Sales Manager London ****k ****...,London,permanent,33000
1,An ideal opportunity for an individual that ha...,London,permanent,50000
2,Online Content and Brand Manager// Luxury Reta...,South East London,permanent,40000
3,A great local marketleader is seeking a perman...,Dereham,permanent,22500
4,Registered Nurse / RGN Nursing Home for Young...,Sutton Coldfield,NaN,20355


In [52]:
fullDesc = data.FullDescription.map(lambda x: x.lower()).replace('[^a-zA-Z0-9]', ' ', regex = True)
locNorm = data.LocationNormalized.map(lambda x: x.lower())
locNorm.fillna('nan', inplace=True)
data.ContractTime.fillna('nan', inplace=True)
X_train = pandas.concat([fullDesc, locNorm, data.ContractTime, data.SalaryNormalized],axis=1)
X_train.head()

,FullDescription,LocationNormalized,ContractTime,SalaryNormalized
0,international sales manager london k ...,london,permanent,33000
1,an ideal opportunity for an individual that ha...,london,permanent,50000
2,online content and brand manager luxury reta...,south east london,permanent,40000
3,a great local marketleader is seeking a perman...,dereham,permanent,22500
4,registered nurse rgn nursing home for young...,sutton coldfield,nan,20355


In [53]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfv = TfidfVectorizer(min_df=5)
X = tfv.fit_transform(X_train['FullDescription'])

In [54]:
from sklearn.feature_extraction import DictVectorizer

enc = DictVectorizer()
X_train_categ = enc.fit_transform(X_train[['LocationNormalized', 'ContractTime']].to_dict('records'))


In [58]:
from scipy.sparse import hstack

U = hstack(blocks=[X,X_train_categ]).toarray()
print(U)

[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


In [60]:
from sklearn.linear_model import Ridge

clf = Ridge(alpha=1,random_state=241)
y = data['SalaryNormalized'].copy()
clf.fit(U,y)

Ridge(alpha=1, copy_X=True, fit_intercept=True, max_iter=None,
   normalize=False, random_state=241, solver='auto', tol=0.001)

In [80]:
test_data = pandas.read_csv('salary-test-mini.csv')
test_data.loc[0,'LocationNormalized'] = test_data.LocationNormalized[0].lower()
test_data.loc[1,'LocationNormalized'] = test_data.LocationNormalized[1].lower()
test_data.FullDescription = test_data.FullDescription.map(lambda x: x.lower()).replace('[^a-zA-Z0-9]', ' ', regex = True)
test_data.head()

,FullDescription,LocationNormalized,ContractTime,SalaryNormalized
0,we currently have a vacancy for an hr project ...,milton keynes,contract,NaN
1,a web developer opportunity has arisen with an...,manchester,permanent,NaN


In [83]:
testFD = tfv.transform(test_data['FullDescription'])
test_categ = enc.transform(test_data[['LocationNormalized', 'ContractTime']].to_dict('records'))
cor = hstack(blocks=[testFD,test_categ]).toarray()
cor

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [84]:
prediction = clf.predict(cor)
prediction # 53625.03 36313.50

array([53625.03261793, 36313.49741836])